In [13]:
import numpy as np
print(np.argmax(np.bincount([0,0,1],weights=[0.2,0.2,0.6])))

ex = np.array([[0.9,0.1],
               [0.8,0.2],
               [0.4,0.6]])
p = np.average(ex, axis=0,weights=[0.2,0.2,0.6])
print(p)
print(np.argmax(p))

1
[ 0.58  0.42]
0


In [14]:
# お試し
print(np.bincount([0,0,2]))
print(np.bincount([0,0,1],weights=[0.2,0.2,0.6]))
print(np.bincount([0,0,2,2],weights=[0.2,0.2,0.6,0.5]))

[2 0 1]
[ 0.4  0.6]
[ 0.4  0.   1.1]


MajorityVoteClassifier

In [27]:
# MajorityVoteClassifierクラス
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator

class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, classifiers, vote='classlabel', weights=None):
        self.classifiers = classifiers
        self.named_classifiers = {key: value for key , value in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights
    
    def fit(self ,X,y):
        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
        for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self
    
    def predict(self, X):
        if self.vote == 'probability':
            maj_vote - np.argmax(self.predict_proba(X), axis=1)
        else:
            predictions = np.asarray([clf.predict(X) for clf in self.classifiers_]).T
            maj_vote = np.apply_along_axis(lambda x:np.argmax(np.bincount(x, weights=self.weights)), axis=1, arr=predictions)
            
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
        return maj_vote
    
    def predict_proba(self, X):
        probas = np.asarray([clf.predict_proba(X) for clf in self.classifiers_])
        avg_proba = np.average(probas, axis=0, weights=self.weights)
        return avg_proba
    
    def get_params(self, deep=True):
        if not deep:
            return super(MajorityVoteClassifier, self).get_params(deep=False)
        else:
            out = self.named_classifiers.copy()
            for name , step in six.iteritems(self.named_classifiers):
                for key, value in six.iteritems(step.get_params(deep=True)):
                    out['%s__%s' % (name,key)] = value
            return out
    


In [28]:
#データ取得
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
iris = datasets.load_iris()
X, y = iris.data[50:, [1,2]],iris.target[50:]
le=LabelEncoder()
y= le.fit_transform(y)

X_train, X_test, y_train,y_test = train_test_split(X, y, test_size=0.5, random_state=1) 

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import numpy as np

clf1 = LogisticRegression(penalty='l2', C=0.001,random_state=0)
clf2 = DecisionTreeClassifier(max_depth=1, criterion='entropy', random_state=0)
clf3 = KNeighborsClassifier(n_neighbors=1, p=2, metric='minkowski')

pipe1 = Pipeline([['sc', StandardScaler()], ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()], ['clf', clf3]])
clf_labels = ['Logisitic Regression', 'Decision Tree', 'KNN']
print('10-fold cross validation:\n')
for clf, label in zip([pipe1, clf2, pipe3], clf_labels):
    scores = cross_val_score(estimator= clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print('ROC AUC: %0.3f (+/- %0.3f) [%s]' %(scores.mean(), scores.std(), label))

10-fold cross validation:

ROC AUC: 0.917 (+/- 0.201) [Logisitic Regression]
ROC AUC: 0.917 (+/- 0.154) [Decision Tree]
ROC AUC: 0.933 (+/- 0.104) [KNN]


In [32]:
mv_clf = MajorityVoteClassifier(classifiers=[pipe1, clf2, pipe3])
clf_labels += ['Majority Voting']
all_clf = [pipe1, clf2, pipe3, mv_clf]
for clf, label in zip(all_clf, clf_labels):
    scores = cross_val_score(estimator= clf,X=X_train,y=y_train,cv=10,scoring='roc_auc')
    print('ROC AUC: %0.3f (+/- %0.3f) [%s]' %(scores.mean(), scores.std(), label))   

ROC AUC: 0.917 (+/- 0.201) [Logisitic Regression]
ROC AUC: 0.917 (+/- 0.154) [Decision Tree]
ROC AUC: 0.933 (+/- 0.104) [KNN]
ROC AUC: 0.967 (+/- 0.100) [Majority Voting]
